In [1]:
import pandas as pd
import numpy as np
import yaml
import os
from geopy.geocoders import Nominatim
from tqdm import tqdm
from glob import glob

In [419]:
## matchid,venue,country,date,month,year,match_between,toss_winner,toss_decision,
##current_innings,over,batsman,non_striker,bowler,extras,total_runs,batsman_runs,extras_type
##out,out_batsman,dismissal_type,dismissed_by_fielder,
##winner,win_by,method_if_any,super_over,super_over_winner,potm

df_data = []

def get_country(venue):
    
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(venue,language="en")
    country = location.address.split(",")[-1].strip()
    return country


def create_csv(folder_path):
    
   
    for file in tqdm(os.listdir(folder_path)):
        #print(file)
        try:
            with open(folder_path+file) as stream:
                data = yaml.safe_load(stream)

                ####### Static Data#####
                match_id = file.split(".")[0]
                try:
                    venue_city = data["info"]["city"]
                except:    
                    venue_city = data["info"]["venue"]
                #print(venue_city)
                try:
                    country = get_country(venue_city)
                except:
                    country = "None"
                print(file,venue_city,country)
                #print(data["info"]["dates"][0],str(data["info"]["dates"][0]).split("-"))
                year,month,day = str(data["info"]["dates"][0]).split("-")
                if "eliminator" in list(data["info"]["outcome"].keys()):
                    #super_over = "Yes"
                    super_over_winner_team = data["info"]["outcome"]["eliminator"]
                    winner_team = "Tie"
                    winner_method = "Tie"
                    winner_num = "Tie"

                elif "result" in list(data["info"]["outcome"].keys()):
                    if data["info"]["outcome"] == "no result":

                        winner_team = "No Result"
                        winner_method = "No Result"
                        winner_num = "No Result"
                        super_over_winner_team = "None"

                else:
                    winner_team = data["info"]["outcome"]["winner"]
                    if "by" in list(data["info"]["outcome"].keys()):
                        winner_method = list(data["info"]["outcome"]["by"].keys())[0]
                        winner_num = list(data["info"]["outcome"]["by"].values())[0]
                    else:
                        ###Awarded Methods###
                        winner_method = data["info"]["outcome"]["method"]
                        winner_num = "None"
                    #super_over = "No"
                    super_over_winner_team = "None"

                match_between = data["info"]["teams"][0] + " vs " + data["info"]["teams"][1]
                toss_winner = data["info"]["toss"]["winner"]
                toss_decision = data["info"]["toss"]["decision"]
                try:
                    potm = "/".join((data["info"]["player_of_match"]))
                except:
                    potm = "No Data"

                if "method" in data["info"]["outcome"].keys():
                    method_if_any = data["info"]["outcome"]["method"]
                else:
                    method_if_any = "None"
                #############################


                for inn in data["innings"]:
                    innings = list(inn.values())[0]
                    #print(innings)
                    current_innings_team = innings["team"]
                    if current_innings_team == data["info"]["teams"][0]:
                        bowling_team = data["info"]["teams"][1]
                    elif current_innings_team == data["info"]["teams"][1]:
                        bowling_team = data["info"]["teams"][0]
                    
                    if "super over" in list(inn.keys())[0].lower():
                        super_over = "Yes"
                    else:
                        super_over = "No"
                    for delivery in list(inn.values())[0]["deliveries"]:
                        #print(delivery)
                        over_number = list(delivery.keys())[0]
                        over_values = list(delivery.values())[0]
                        batsman = over_values["batsman"]
                        non_striker = over_values["non_striker"]
                        bowler = over_values["bowler"]
                        extras,total_runs,batsman_runs = over_values["runs"]["extras"],over_values["runs"]["total"],over_values["runs"]["batsman"]
                        if extras > 0:
                            extras_type = list(over_values["extras"].keys())[0]
                        else:
                            extras_type = "None"

                        if "wicket" in over_values.keys():
                            out = "Yes"
                            player_out = over_values["wicket"]["player_out"]
                            out_type = over_values["wicket"]["kind"]
                            if "fielders" in over_values["wicket"]:
                                dismissed_by_fielder = "/".join((over_values["wicket"]["fielders"]))
                            else:
                                dismissed_by_fielder = "None"
                        else:
                            out = "No"
                            player_out = "None"
                            out_type = "None"
                            dismissed_by_fielder = "None"

                        df_data.append([match_id,venue_city,country,day,month,year,match_between,toss_winner,toss_decision,\
                       current_innings_team,bowling_team,over_number,batsman,non_striker,bowler,extras,total_runs,batsman_runs,\
                                    extras_type,\
                        out,player_out,out_type,dismissed_by_fielder,winner_team,winner_method,winner_num,method_if_any,\
                                        super_over,super_over_winner_team,potm])


        #return df_data
        except TypeError as e:
            print("passed")
            pass

In [425]:
create_csv("/home/shivargha/performance_prediction/odi_cricket/odis/")
df = pd.DataFrame(df_data,
                  columns=["matchid","venue","country","date","month","year","match_between",
                           "toss_winner","toss_decision",\
                          "current_innings","bowling_innings","over","batsman","non_striker","bowler","extras",\
                           "total_runs","batsman_runs",\
                          "extras_type","out","out_batsman","dismissal_type","dismissed_by_fielder",\
                          "winner","winner_method","win_by","method_if_any","super_over","super_over_winner","potm" 
                          ])

  0%|                                          | 1/2106 [00:00<34:34,  1.01it/s]

657643.yaml Abu Dhabi United Arab Emirates


  0%|                                          | 2/2106 [00:02<39:35,  1.13s/it]

317581.yaml Bulawayo Zimbabwe


  0%|                                          | 3/2106 [00:03<37:53,  1.08s/it]

...

516209.yaml Colombo Sri Lanka


100%|██████████████████████████████████████▉| 2105/2106 [43:36<00:00,  1.03it/s]

1079248.yaml Harare Sports Club Zimbabwe


100%|███████████████████████████████████████| 2106/2106 [43:37<00:00,  1.24s/it]

1262347.yaml Dhaka Bangladesh


In [426]:
df.to_csv("/home/shivargha/performance_prediction/odi_cricket/odi.csv")

In [1]:
##My Analysis###
import pandas as pd
import numpy as np
import yaml
import os
from geopy.geocoders import Nominatim
from tqdm import tqdm
from glob import glob
df = pd.read_csv("/home/shivargha/performance_prediction/odi_cricket/odi.csv",index_col=0,low_memory=False)

In [2]:
df.columns

Index(['matchid', 'venue', 'country', 'date', 'month', 'year', 'match_between',
       'toss_winner', 'toss_decision', 'current_innings', 'bowling_innings',
       'over', 'batsman', 'non_striker', 'bowler', 'extras', 'total_runs',
       'batsman_runs', 'extras_type', 'out', 'out_batsman', 'dismissal_type',
       'dismissed_by_fielder', 'winner', 'winner_method', 'win_by',
       'method_if_any', 'super_over', 'super_over_winner', 'potm'],
      dtype='object')

In [46]:
##Best batsman for australia in australia vs england in australia
#len(df_aus_auseng["matchid"].unique())

def create_df_team1_v_team2(df,team1,team2,venue,country):
    
    ###team1 - home
    ##team2 - away
    
    if (venue is not None) & (country is None):
        df1 = df.loc[(df["venue"] == venue ) & (df["match_between"] == str(team1)+" vs "+str(team2))]
        return df1
    elif (venue is not None) & (country is not None):
        df1 = df.loc[(df["venue"] == venue) & (df["country"] == country) & (df["match_between"] == str(team1)+" vs "+str(team2))]
        return df1
    elif (venue is None) & (country is not None):
        df1 = df.loc[(df["country"] == country ) & (df["match_between"] == str(team1)+" vs "+str(team2))]
        return df1
    else:
        df1 = df.loc[(df["match_between"] == str(team1)+" vs "+str(team2))]
        return df1

    

def get_batsmen_of_team(team,df):
    ###Get batsman name of a certain country#
    batsmen_list = list(df.loc[df["current_innings"]==team]["batsman"].unique())
    return batsmen_list

def get_bowler_of_team(team,df):
    
    bowler_list = list(df.loc[df["bowling_innings"] == team]["bowler"].unique())
    return bowler_list


def batsman_score_stats_vs_team_or_by_venue_country(df,batsman_name,opposition_team,venue,country):
    
    
    scores_dict = dict()
    if (opposition_team is not None) & (venue is not None) & (country is None):
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["bowling_innings"] == opposition_team) \
                    & (df["venue"] == venue )]
        
    elif (opposition_team is not None) & (venue is not None) & (country is not None):
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["bowling_innings"] == opposition_team) \
                     & (df["venue"] == venue) & (df["country"] == country)]
        
    elif (opposition_team is not None) & (venue is None) & (country is not None):  
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["bowling_innings"] == opposition_team) \
                     & (df["country"] == country)]
        
    elif (opposition_team is None) & (venue is not None) & (country is None):
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["venue"] == venue)]
        
    elif (opposition_team is None) & (venue is  None) & (country is not None):
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["country"] == country)]
    
    else :
        df1 = df.loc[(df["batsman"] == batsman_name)]
        
    
    for matchid_no in df1["matchid"].unique():
        df2 = df1.loc[(df1["matchid"] == matchid_no)]
        date_of_match = str(df2.iloc[0]["date"]) + "-" + str(df2.iloc[0]["month"])+"-"+str(str(df2.iloc[0]["year"]))
        balls_faced = df2.shape[0]
        venue_city_stadium = df2.iloc[0]["venue"]
        venue_country = df2.iloc[0]["country"]
        if df2.loc[df2["out"] == "Yes"].shape[0] != 0:

            dissmisal_type = df2.loc[df2["out"] == "Yes"]["dismissal_type"].values[0]
            dissmissed_by = df2.loc[df2["out"] == "Yes"]["bowler"].values[0]
        else:
            dissmisal_type = "Not Out"
            dissmissed_by = "None"

        num_fours = df2.loc[df["batsman_runs"] == 4].shape[0]
        num_sixes = df2.loc[df["batsman_runs"] == 6].shape[0]
        match_between = df2.iloc[0]["match_between"]
        scores_dict[matchid_no] = {"Match_date":date_of_match,"Venue":venue_city_stadium,"Country":venue_country\
                                   ,"Match_between":match_between\
                                   ,"Runs":sum(df2["batsman_runs"]),\
                                   "Balls":balls_faced,"Dismissed By":dissmissed_by,\
                                   "Dismissal_type":dissmisal_type,"Fours":num_fours,\
                                   "Sixes":num_sixes}
    return scores_dict
        
    

In [47]:
def all_batsman_scores_vs_team(df,batsman_team,opposition_team,venue,country):
    

    #print(df)
    if (venue is not None) & (country is None):
        df1 = df.loc[(df["current_innings"] == batsman_team) & (df["bowling_innings"] == opposition_team) & \
                     (df["venue"] == venue)]
    elif (venue is not None) & (country is not None):
        df1 = df.loc[(df["current_innings"] == batsman_team) & (df["bowling_innings"] == opposition_team) & \
                     (df["venue"] == venue) \
                    & df["country"] == country]
    elif (venue is None) & (country is not None):
        df1 = df.loc[(df["current_innings"] == batsman_team) & (df["bowling_innings"] == opposition_team) \
                    & df["country"] == country]
    else:
        df1 = df.loc[(df["current_innings"] == batsman_team) & (df["bowling_innings"] == opposition_team)]
        
    scores_dict = dict()
    df = df1
    
    for batsman_name in df["batsman"].unique():
        df1 = df.loc[(df["batsman"] == batsman_name)]
        match_dict = dict()
        for matchid_no in df1["matchid"].unique():
            df2 = df1.loc[(df1["matchid"] == matchid_no)]
            date_of_match = str(df2.iloc[0]["date"]) + "-" + str(df2.iloc[0]["month"])+"-"+str(str(df2.iloc[0]["year"]))
            balls_faced = df2.shape[0]
            venue_city_stadium = df2.iloc[0]["venue"]
            venue_country = df2.iloc[0]["country"]
            if df2.loc[df2["out"] == "Yes"].shape[0] != 0:

                dissmisal_type = df2.loc[df2["out"] == "Yes"]["dismissal_type"].values[0]
                dissmissed_by = df2.loc[df2["out"] == "Yes"]["bowler"].values[0]
            else:
                dissmisal_type = "Not Out"
                dissmissed_by = "None"

            num_fours = df2.loc[df["batsman_runs"] == 4].shape[0]
            num_sixes = df2.loc[df["batsman_runs"] == 6].shape[0]

            match_dict[matchid_no] = {"Match_date":date_of_match,"Venue":venue_city_stadium,"Country":venue_country\
                                       ,"Runs":sum(df2["batsman_runs"]),\
                                       "Balls":balls_faced,"Dismissed By":dissmissed_by,\
                                       "Dismissal_type":dissmisal_type,"Fours":num_fours,\
                                       "Sixes":num_sixes}


        scores_dict[batsman_name] = match_dict
    return scores_dict
        

In [30]:
###Head to Head Analysis##
###Batsman v Bowler###
from collections import Counter
def batsman_v_bowler_h2h(df,batsman_name,bowler_name,venue,country):
    
    head_2_head_dict = dict() #####Match by Match#####
    overall_head2head = dict() ###Overall Head to Head#
    if (venue is not None) & (country is None):
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["bowler"] == bowler_name) & (df["venue"] == venue)]
    elif (venue is not None) & (country is not None):
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["bowler"] == bowler_name) & (df["venue"] == venue) \
                    & df["country"] == country]
    elif (venue is None) & (country is not None):
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["bowler"] == bowler_name) \
                    & df["country"] == country]
    else:
        df1 = df.loc[(df["batsman"] == batsman_name) & (df["bowler"] == bowler_name)]
    
    
        
    head_to_head = str(batsman_name)+" vs "+str(bowler_name)
    matches = len(list(df1["matchid"].unique()))


    total_runs = sum(df1["batsman_runs"])
    total_balls = df1.shape[0]
    try:
        total_fours = dict(Counter(df1["batsman_runs"]))[4]
    except KeyError as e:
        total_fours = 0
    try:
        total_sixes = dict(Counter(df1["batsman_runs"]))[6]
    except KeyError as e:
        total_sixes = 0
    try:
        num_out = dict(Counter(df1["out"]))["Yes"]
        dissmisal_dict = dict() 
        for i in dict(Counter(df1["dismissal_type"])):

            if i != "None":
                dissmisal_type1 = i
                dissmisal_type_number = dict(Counter(df1["dismissal_type"]))[i]
                dissmisal_dict[dissmisal_type1] = dissmisal_type_number
    except KeyError as e:
        num_out = 0
        dissmisal_dict = "None"

    overall_head2head[head_to_head] = {"Num Matches":matches,"Total Runs":total_runs,"Total Balls":total_balls\
                                       ,"Total 4s":total_fours,\
                                          "Total 6s":total_sixes,"Total Outs":num_out,\
                                          "Dissmisal Types":dissmisal_dict}


    for matchid_no in df1["matchid"].unique():

        df2 = df1.loc[(df1["matchid"] == matchid_no)]
        date_of_match = str(df2.iloc[0]["date"]) + "-" + str(df2.iloc[0]["month"])+"-"+str(str(df2.iloc[0]["year"]))
        venue_city_stadium = df2.iloc[0]["venue"]
        venue_country = df2.iloc[0]["country"]
        balls_faced = df2.shape[0]
        if df2.loc[df2["out"] == "Yes"].shape[0] != 0:

                dissmisal_type = df2.loc[df2["out"] == "Yes"]["dismissal_type"].values[0]
                dissmissed_by = df2.loc[df2["out"] == "Yes"]["bowler"].values[0]
        else:
            dissmisal_type = "Not Out"
            dissmissed_by = "None"

        num_fours = df2.loc[df["batsman_runs"] == 4].shape[0]
        num_sixes = df2.loc[df["batsman_runs"] == 6].shape[0]


        head_2_head_dict[matchid_no] = {"Match_date":date_of_match,"Venue":venue_city_stadium,\
                                        "Country":venue_country\
                                       ,"Runs":sum(df2["batsman_runs"]),\
                                       "Balls":balls_faced,"Dismissed By":dissmissed_by,\
                                       "Dismissal_type":dissmisal_type,"Fours":num_fours,\
                                       "Sixes":num_sixes}
        
    return head_2_head_dict,overall_head2head

In [45]:
df.columns

Index(['matchid', 'venue', 'country', 'date', 'month', 'year', 'match_between',
       'toss_winner', 'toss_decision', 'current_innings', 'bowling_innings',
       'over', 'batsman', 'non_striker', 'bowler', 'extras', 'total_runs',
       'batsman_runs', 'extras_type', 'out', 'out_batsman', 'dismissal_type',
       'dismissed_by_fielder', 'winner', 'winner_method', 'win_by',
       'method_if_any', 'super_over', 'super_over_winner', 'potm'],
      dtype='object')

In [66]:
def all_bowler_figures_vs_team_or_by_venue_country(df,bowler_name,opposition_team,venue,country):

    
    scores_dict = dict()
    if (opposition_team is not None) & (venue is not None) & (country is None):
        df1 = df.loc[(df["bowler"] == bowler_name) & (df["current_innings"] == opposition_team) \
                    & (df["venue"] == venue )]
        
    elif (opposition_team is not None) & (venue is not None) & (country is not None):
        df1 = df.loc[(df["bowler"] == bowler_name) & (df["current_innings"] == opposition_team) \
                     & (df["venue"] == venue) & (df["country"] == country)]
        
    elif (opposition_team is not None) & (venue is None) & (country is not None):  
        df1 = df.loc[(df["bowler"] == bowler_name) & (df["current_innings"] == opposition_team) \
                     & (df["country"] == country)]
        
    elif (opposition_team is None) & (venue is not None) & (country is None):
        df1 = df.loc[(df["bowler"] == bowler_name) & (df["venue"] == venue)]
        
    elif (opposition_team is None) & (venue is  None) & (country is not None):
        df1 = df.loc[(df["bowler"] == bowler_name) & (df["country"] == country)]
    
    else :
        df1 = df.loc[(df["bowler"] == bowler_name)]
        
   
    for matchid_no in df1["matchid"].unique():
        df2 = df1.loc[(df1["matchid"] == matchid_no)]
        date_of_match = str(df2.iloc[0]["date"]) + "-" + str(df2.iloc[0]["month"])+"-"+str(str(df2.iloc[0]["year"]))
        balls_bowled = df2.shape[0]
        runs_to_batsman = sum(df2["batsman_runs"]) 
        venue_city_stadium = df2.iloc[0]["venue"]
        venue_country = df2.iloc[0]["country"]
        extras = sum(df2["extras"])
        if df2.loc[df2["out"] == "Yes"].shape[0] != 0:
            number_of_wickets = 0
            dissmissed_batsman = dict()
            for batsman in df2.loc[df2["out"] == "Yes"]["out_batsman"].unique():
                if df2.loc[(df2["out"] == "Yes") & (df2["out_batsman"] == batsman)]["dismissal_type"].values[0] != "run out":
                    number_of_wickets = number_of_wickets + 1
                dissmissed_batsman[batsman] = df2.loc[(df2["out"] == "Yes") & (df2["out_batsman"] == batsman)]["dismissal_type"].values[0]
            if number_of_wickets > 0:
                average = (runs_to_batsman)/(number_of_wickets)
                strike_rate = (balls_bowled)/(number_of_wickets)
            #dissmisal_type = df2.loc[df2["out"] == "Yes"]["dismissal_type"].values[0]
            #dissmissed_by = df2.loc[df2["out"] == "Yes"]["bowler"].values[0]
        else:
            number_of_wickets = 0
            dissmissed_batsman = dict()
            average = "None"
            strike_rate = "None"
            #dissmisal_type = "Not Out"
            #dissmissed_by = "None"

        num_fours = df2.loc[df["batsman_runs"] == 4].shape[0]
        num_sixes = df2.loc[df["batsman_runs"] == 6].shape[0]
        match_between = df2.iloc[0]["match_between"]
        scores_dict[matchid_no] = {"Match_date":date_of_match,"Venue":venue_city_stadium,"Country":venue_country\
                                   ,"Match_between":match_between,"Runs given":runs_to_batsman
                                   ,"Wickets":number_of_wickets,\
                                   "Balls":balls_bowled,\
                                   "Dissmisals":dissmissed_batsman,"Average":average,"Strike Rate":strike_rate,\
                                   "Extras":extras,
                                    "Fours":num_fours,\
                                   "Sixes":num_sixes}
    return scores_dict

In [68]:
df.columns

Index(['matchid', 'venue', 'country', 'date', 'month', 'year', 'match_between',
       'toss_winner', 'toss_decision', 'current_innings', 'bowling_innings',
       'over', 'batsman', 'non_striker', 'bowler', 'extras', 'total_runs',
       'batsman_runs', 'extras_type', 'out', 'out_batsman', 'dismissal_type',
       'dismissed_by_fielder', 'winner', 'winner_method', 'win_by',
       'method_if_any', 'super_over', 'super_over_winner', 'potm'],
      dtype='object')

In [76]:
def all_bowler_figures_vs_team(df,bowler_team,opposition_team,venue,country):
    

    #print(df)
    if (venue is not None) & (country is None):
        df1 = df.loc[(df["current_innings"] == opposition_team) & (df["bowling_innings"] == bowler_team) & \
                     (df["venue"] == venue)]
    elif (venue is not None) & (country is not None):
        df1 = df.loc[(df["current_innings"] == opposition_team) & (df["bowling_innings"] == bowler_team) & \
                     (df["venue"] == venue) \
                    & df["country"] == country]
    elif (venue is None) & (country is not None):
        df1 = df.loc[(df["current_innings"] == opposition_team) & (df["bowling_innings"] == bowler_team) \
                    & df["country"] == country]
    else:
        df1 = df.loc[(df["current_innings"] == opposition_team) & (df["bowling_innings"] == bowler_team)]
        
    scores_dict = dict()
    df = df1
    
    for bowler_name in df["bowler"].unique():
        df1 = df.loc[(df["bowler"] == bowler_name)]
        match_dict = dict()
        for matchid_no in df1["matchid"].unique():
            df2 = df1.loc[(df1["matchid"] == matchid_no)]
            date_of_match = str(df2.iloc[0]["date"]) + "-" + str(df2.iloc[0]["month"])+"-"+str(str(df2.iloc[0]["year"]))
            balls_bowled = df2.shape[0]
            runs_to_batsman = sum(df2["batsman_runs"]) 
            venue_city_stadium = df2.iloc[0]["venue"]
            venue_country = df2.iloc[0]["country"]
            extras = sum(df2["extras"])
            if df2.loc[df2["out"] == "Yes"].shape[0] != 0:
                number_of_wickets = 0
                dissmissed_batsman = dict()
                for batsman in df2.loc[df2["out"] == "Yes"]["out_batsman"].unique():
                    if df2.loc[(df2["out"] == "Yes") & (df2["out_batsman"] == batsman)]["dismissal_type"].values[0] != "run out":
                        number_of_wickets = number_of_wickets + 1
                    dissmissed_batsman[batsman] = df2.loc[(df2["out"] == "Yes") & (df2["out_batsman"] == batsman)]["dismissal_type"].values[0]
                if number_of_wickets > 0:
                    average = (runs_to_batsman)/(number_of_wickets)
                    strike_rate = (balls_bowled)/(number_of_wickets)
                #dissmisal_type = df2.loc[df2["out"] == "Yes"]["dismissal_type"].values[0]
                #dissmissed_by = df2.loc[df2["out"] == "Yes"]["bowler"].values[0]
            else:
                number_of_wickets = 0
                dissmissed_batsman = dict()
                average = "None"
                strike_rate = "None"
                #dissmisal_type = "Not Out"
                #dissmissed_by = "None"

            num_fours = df2.loc[df["batsman_runs"] == 4].shape[0]
            num_sixes = df2.loc[df["batsman_runs"] == 6].shape[0]
            match_between = df2.iloc[0]["match_between"]
            match_dict[matchid_no] = {"Match_date":date_of_match,"Venue":venue_city_stadium,"Country":venue_country\
                                   ,"Match_between":match_between,"Runs given":runs_to_batsman
                                   ,"Wickets":number_of_wickets,\
                                   "Balls":balls_bowled,\
                                   "Dissmisals":dissmissed_batsman,"Average":average,"Strike Rate":strike_rate,\
                                      "Extras":extras,
                                    "Fours":num_fours,\
                                   "Sixes":num_sixes}


        scores_dict[bowler_name] = match_dict
    return scores_dict
        

In [77]:
all_bowler_figures_vs_team(df,"New Zealand","India",None,None)

{'SE Bond': {218250: {'Match_date': '6-9-2005',
   'Venue': 'Harare Sports Club',
   'Country': 'Zimbabwe',
   'Match_between': 'India vs New Zealand',
   'Runs given': 40,
   'Wickets': 1,
   'Balls': 61,
   'Dissmisals': {'AB Agarkar': 'caught'},
   'Average': 40.0,
   'Strike Rate': 61.0,
   'Extras': 4,
   'Fours': 5,
   'Sixes': 0},
  403382: {'Match_date': '11-9-2009',
   'Venue': 'Colombo',
   'Country': 'Sri Lanka',
   'Match_between': 'India vs New Zealand',
   'Runs given': 29,
   'Wickets': 0,
   'Balls': 61,
   'Dissmisals': {},
   'Average': 'None',
   'Strike Rate': 'None',
   'Extras': 2,
   'Fours': 4,
   'Sixes': 0}},
 'KD Mills': {218250: {'Match_date': '6-9-2005',
   'Venue': 'Harare Sports Club',
   'Country': 'Zimbabwe',
   'Match_between': 'India vs New Zealand',
   'Runs given': 45,
   'Wickets': 2,
   'Balls': 57,
   'Dissmisals': {'Yuvraj Singh': 'caught', 'A Nehra': 'caught'},
   'Average': 22.5,
   'Strike Rate': 28.5,
   'Extras': 1,
   'Fours': 7,
   'Sixes